In [1]:
import os
import numpy as np
import pandas as pd
from pandas import DataFrame
import pickle

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline

# Allows more columns to be shown for outputs
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5200)

In [2]:
#set directories
dirRawData = "C:/Users/kevin/Google Drive/AML/Projects/Coursework/RawData/"
dirPData = "C:/Users/kevin/Google Drive/AML/Projects/Coursework/PData/"
dirPOutput = "C:/Users/kevin/Google Drive/AML/Projects/Coursework/POutput/"

In [3]:
#import variables
f_name = dirPData + 'prepVars.pickle'

with open(f_name, "rb") as f:
    dict_ = pickle.load(f)
    
df_test = dict_['df_test']
df_train = dict_['df_train']
df_train_num = dict_['df_train_num']
df_train_cat = dict_['df_train_cat']
vars_ind_numeric_train = dict_['vars_ind_numeric_train']
vars_ind_cat_train = dict_['vars_ind_cat_train']
del f_name, dict_

In [4]:
train_y = df_train['target'].values

var_notToUse = ['id']

allFeatures = [var for var in df_train if var not in 'target'] 
allFeaturesNum = list(df_train[allFeatures].columns[df_train[allFeatures].dtypes != 'object'])

train_X = df_train[allFeaturesNum]
test_X = df_test[allFeaturesNum]

# Validation scheme
* We split the train data into a 75/25 split between train/test. The model is first trained onto the split train data (train_Xv and train_yv) and then the model is used to predict the probability estimates of the test data (test Xv). We then use the roc auc score (Area Under the Receiver Operating Characteristic Curve) as it is the scoring method used by the Kaggle competition.
* Once the score is calculated, the model is trained again on the whole train data (train_X and train_y) without the split and is used to predict the test data (test_X). This is then submitted to Kaggle if the performance is satisfactory.

In [5]:
#prepare validation data
train_Xv, test_Xv, train_yv, test_yv = train_test_split(train_X, train_y, test_size=0.25, random_state=1)

## Random Forest Classifier Model
### Kaggle Score: 0.83528, Validation Score: 0.8431343241688068
<img src="naiverfc1.png">
* Naive model using only numeric variables
* assume all variables explain target
* This model outputs probabilities of 1s
* set the seed for reproducible results
* set the number of trees to 100

In [6]:
randForest_rModelv = RandomForestClassifier(random_state=1, n_estimators=100)
randForest_rModelv.fit(train_Xv, train_yv)
randfprediction1v = randForest_rModelv.predict_proba(test_Xv)
print("Validation score:",roc_auc_score(test_yv, randfprediction1v[:,1]))

Validation score: 0.8431343241688068


In [7]:
randForest_rModel = RandomForestClassifier(random_state=1, n_estimators=100)
randForest_rModel.fit(train_X, train_y)
prediction1 = randForest_rModel.predict_proba(test_X)

pred1 = {'id': df_test['id'].values,
        'target': prediction1[:,1]}
df_pred1 = DataFrame(pred1)

df_pred1 = df_pred1.reset_index().set_index('id')
df_pred1 = df_pred1.drop(columns='index')
df_pred1.head()

,target
id,
5,0.06
10,0.27
11,0.17
13,0.40
16,0.09


In [8]:
df_pred1.to_csv(r'C:/Users/kevin/Google Drive/AML/Projects/Coursework/POutput/naiverfc1.csv')